In [1]:
# Install required libraries
import os

try:
    import selenium
except ImportError:
    os.system("pip install selenium")

try:
    import webdriver_manager
except ImportError:
    os.system("pip install webdriver-manager")

try:
    import pandas as pd
except ImportError:
    os.system("pip install pandas")

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import pandas as pd
import time

In [2]:
# Initialize Edge with options
edge_options = Options()
edge_options.add_argument("--start-maximized")       # Start maximized for better visibility
edge_options.add_argument("--disable-infobars")      # Disable infobars
edge_options.add_argument("--disable-extensions")    # Disable extensions for simplicity

# Use Edge driver
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)

def scroll_to_bottom(driver, pause_time=2):
    """
    Scroll to the bottom of the page to load dynamic content.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.get("https://www.expedia.com/Hotel-Search?adults=2&children=&d1=2024-12-13&d2=2024-12-18&destination=Lisbon%2C%20Lisbon%20District%2C%20Portugal&endDate=2024-12-19&latLong=38.7123%2C-9.13833&lodging=HOTEL&mapBounds=&regionId=2080&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2024-12-14&theme=&useRewards=false&userIntent=")
time.sleep(6)  # Wait for the page to load completely

# Extract hotel data
listing_container = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="property-listing-results"]'))
)
hotel_links = listing_container.find_elements(By.CSS_SELECTOR, 'a[data-stid="open-hotel-information"]')
print(f"Found {len(hotel_links)} hotel links.")


Found 350 hotel links.


In [3]:
import pandas as pd

# Initialize a list to store all hotel data
hotel_data = []
location = "Lisbon, Portugal"

for index, link in enumerate(hotel_links):
    try:
        href = link.get_attribute('href')
        print(f"\nProcessing link {index + 1}")

        # Open the link in a new tab using JavaScript
        driver.execute_script("window.open(arguments[0]);", href)
        time.sleep(3)  # Wait for the new tab to open

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(15)  # Wait for the new page to load

        hotel_details = {"Location": location}

        # Extract Hotel Name
        try:
            hotel_name_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-3"))
            )
            hotel_details['Hotel Name'] = hotel_name_element.text.strip()
        except Exception as e:
            print(f"Error extracting hotel name: {e}")
            hotel_details['Hotel Name'] = None

        # Extract Stars
        try:
            stars_container = driver.find_element(By.CSS_SELECTOR, "div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one")
            stars_count = len(stars_container.find_elements(By.TAG_NAME, "svg"))
            hotel_details['Stars'] = stars_count
        except Exception as e:
            print(f"Error extracting stars for hotel {index + 1}: {e}")
            hotel_details['Stars'] = None

        # Extract Rating
        try:
            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.uitk-badge-base-large.uitk-badge-positive > span.uitk-badge-base-text")
                )
            )
            hotel_details['Rating'] = rating_element.text.strip()
        except Exception as e:
            print(f"Error extracting rating: {e}")
            hotel_details['Rating'] = None

        # Extract Review Keyword
        try:
            review_summary_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-stid='content-hotel-reviewsummary']"))
            )
            review_keyword_element = review_summary_container.find_element(
                By.CSS_SELECTOR, "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"
            )
            review_keyword = review_keyword_element.text.strip()
            hotel_details['Review Keyword'] = review_keyword
            print(f"Extracted Review Keyword: {review_keyword}")
        except Exception as e:
            print(f"Error extracting review keyword: {e}")
            hotel_details['Review Keyword'] = None

        # Extract Services
        try:
            services_list = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.uitk-typelist[data-stid='sp-content-list']"))
            )

            services_lis = services_list.find_elements(By.CSS_SELECTOR, "li[data-stid^='sp-content-item-']")
            service_count = len(services_lis)
            print(f"Number of services: {service_count}")

            services = []
            for i in range(service_count):
                try:
                    service_span = driver.find_element(
                        By.CSS_SELECTOR,
                        f"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-{i}'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"
                    )
                    services.append(service_span.text.strip())
                except Exception as e:
                    print(f"Error extracting service {i}: {e}")
                    services.append(None)

            hotel_details['Services'] = services
        except Exception as e:
            print(f"Error extracting services: {e}")
            hotel_details['Services'] = []

        # Extract Property Offers
        try:
            property_offers_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="section-room-list"]'))
            )

            property_offers = property_offers_container.find_elements(By.CSS_SELECTOR, 'div[data-stid^="property-offer-"]')

            offers = []
            for offer_index, offer in enumerate(property_offers, start=1):
                offer_details = {}

                try:
                    room_name_element = offer.find_element(By.CSS_SELECTOR, "h3.uitk-heading.uitk-heading-6")
                    offer_details['Room Name'] = room_name_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room name for offer {offer_index}: {e}")
                    offer_details['Room Name'] = None

                try:
                    price_summary_div = offer.find_element(By.CSS_SELECTOR, 'div[data-stid="price-summary"]')
                    room_charges_element = price_summary_div.find_element(
                        By.CSS_SELECTOR,
                        "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-emphasis-theme"
                    )
                    offer_details['Room Charges'] = room_charges_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room charges for offer {offer_index}: {e}")
                    offer_details['Room Charges'] = None

                offers.append(offer_details)

            hotel_details['Property Offers'] = offers
        except Exception as e:
            print(f"Error extracting property offers: {e}")
            hotel_details['Property Offers'] = []

        # Add hotel details to list
        hotel_data.append(hotel_details)
        print(f"Extracted Hotel Data: {hotel_details}")

        # Close the new tab
        driver.close()

        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")
        continue

# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)

# Normalize property offers if necessary
if 'Property Offers' in df.columns:
    df['Property Offers'] = df['Property Offers'].apply(lambda x: ', '.join(str(offer) for offer in x) if isinstance(x, list) else x)

# Save to CSV
output_filename = f"hotel_data_{location.replace(',', '').replace(' ', '_')}.csv"
df.to_csv(output_filename, index=False)
print(f"Data exported to {output_filename}")


Processing link 1
Error extracting hotel name: Message: 
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759DC92F7]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symbol) [0x00007FF759D81E30]
	(No symbol) [0x00007FF759D82571]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A13BAF4+1094964]
	(No symbol) [0x00007FF759EA32B8]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A13AF33+1091955]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A13AA99+1090777]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF759F40CA1+461553]
	Microsoft::Applications::Events::ILogConfigur


Processing link 3
Extracted Review Keyword: Exceptional
Number of services: 6
Error extracting room charges for offer 2: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symbol


Processing link 4
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 3: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symbol) 


Processing link 8
Extracted Review Keyword: Exceptional
Number of services: 6
Error extracting room charges for offer 2: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symbol


Processing link 9
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 6: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symbol) 

...

Error extracting room charges for offer 6: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symbol) [0x00007FF759D81E30]
	(No symbol) [0x00007FF759D82571]
	Microsoft::Applicatio


Processing link 232
Extracted Review Keyword: Exceptional
Number of services: 6
Error extracting room charges for offer 3: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symb


Processing link 234
Extracted Review Keyword: Exceptional
Number of services: 6
Extracted Hotel Data: {'Location': 'Lisbon, Portugal', 'Hotel Name': 'Olissippo Lapa Palace – The Leading Hotels of the World', 'Stars': 5, 'Rating': '9.8', 'Review Keyword': 'Exceptional', 'Services': ['Pool', 'Spa', 'Hot Tub', 'Bar', 'Parking available', 'Gym'], 'Property Offers': [{'Room Name': 'Double or Twin Room, Garden View', 'Room Charges': '$412'}, {'Room Name': 'Double Room, River View', 'Room Charges': '$468'}, {'Room Name': 'Superior Room (Palace)', 'Room Charges': '$551'}, {'Room Name': 'Pool Or Garden View Room', 'Room Charges': '$413'}, {'Room Name': 'River View Room', 'Room Charges': '$469'}, {'Room Name': 'Palace Superior Room', 'Room Charges': '$552'}, {'Room Name': 'Deluxe Room (Palace)', 'Room Charges': '$709'}, {'Room Name': 'Junior Suite', 'Room Charges': '$887'}, {'Room Name': 'Suite (Garden)', 'Room Charges': '$1,803'}]}

Processing link 235
Extracted Review Keyword: Exceptional
Num


Processing link 238
Extracted Review Keyword: Exceptional
Number of services: 6
Error extracting room charges for offer 4: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: MicrosoftEdge=131.0.2903.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF759E66B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75A18F464+1437348]
	sqlite3_dbdata_init [0x00007FF75A232DB6+643206]
	(No symbol) [0x00007FF759D8C9DB]
	(No symbol) [0x00007FF759D8CAE3]
	(No symbol) [0x00007FF759D8348C]
	(No symbol) [0x00007FF759DAC1DF]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759D832FD]
	(No symbol) [0x00007FF759DAC480]
	(No symbol) [0x00007FF759D83437]
	(No symbol) [0x00007FF759DC6BFF]
	(No symbol) [0x00007FF759DABE03]
	(No symbol) [0x00007FF759D82984]
	(No symb

Data exported to hotel_data_Lisbon_Portugal.csv
